<a href="https://colab.research.google.com/github/fermarimoraes/sistemaderecomendacao/blob/main/Recomenda%C3%A7%C3%A3o_de_filmes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [11]:
filmes = pd.read_csv('movies.csv')
filmes.columns = ['filmeId', ' títulos', 'generos']
filmes = filmes.set_index('filmeId')
filmes.head()

,títulos,generos
filmeId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [12]:
notas = pd.read_csv('ratings.csv')
notas.columns = ['usuárioID', 'filmeId', 'notas', 'momento']
notas.head()

,usuárioID,filmeId,notas,momento
0,1,1,4.0,1.225735e+09
1,1,110,4.0,1.225865e+09
2,1,158,4.0,1.225734e+09
3,1,260,4.5,1.225735e+09
4,1,356,5.0,1.225735e+09


In [7]:
notas.describe()

,usuárioID,filmeId,notas,momento
count,4.363797e+06,4.363797e+06,4.363797e+06,4.363797e+06
mean,2.132895e+04,2.770880e+04,3.546348e+00,1.267910e+09
std,1.233654e+04,4.930802e+04,1.056359e+00,2.520202e+08
min,1.000000e+00,1.000000e+00,5.000000e-01,1.265139e+06
25%,1.047800e+04,1.219000e+03,3.000000e+00,1.052487e+09
50%,2.136600e+04,3.255000e+03,3.500000e+00,1.257064e+09
75%,3.210500e+04,3.652900e+04,4.000000e+00,1.491373e+09
max,4.261400e+04,2.889570e+05,5.000000e+00,1.689838e+09


#Primeira tentativa de recomendação: Heurística de total de votos.


In [17]:
total_de_votos = notas['filmeId'].value_counts()
total_de_votos.head()

filmeId
318     48346
356     44878
296     43079
2571    42294
593     40337
Name: count, dtype: int64

In [18]:
filmes['total_de_votos'] = total_de_votos
filmes.head()

,títulos,generos,total_de_votos
filmeId,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,30231.0
2,Jumanji (1995),Adventure|Children|Fantasy,12072.0
3,Grumpier Old Men (1995),Comedy|Romance,6272.0
4,Waiting to Exhale (1995),Comedy|Drama|Romance,1174.0
5,Father of the Bride Part II (1995),Comedy,6261.0


In [19]:
filmes.loc[318]

 títulos          Shawshank Redemption, The (1994)
generos                                Crime|Drama
total_de_votos                             48346.0
Name: 318, dtype: object

In [21]:
filmes.sort_values(by='total_de_votos', ascending=False).head()

,títulos,generos,total_de_votos
filmeId,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,48346.0
356,Forrest Gump (1994),Comedy|Drama|Romance|War,44878.0
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,43079.0
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,42294.0
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,40337.0


In [25]:
notas_medias = notas.groupby('filmeId').mean()['notas']

In [30]:
filmes['notas_medias'] = notas_medias
filmes.sort_values(by='total_de_votos', ascending=False).head(10)

,títulos,generos,total_de_votos,notas_medias
filmeId,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,48346.0,4.417232
356,Forrest Gump (1994),Comedy|Drama|Romance|War,44878.0,4.068463
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,43079.0,4.192762
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,42294.0,4.163250
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,40337.0,4.152131
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,38476.0,4.092447
2959,Fight Club (1999),Action|Crime|Drama|Thriller,34022.0,4.241505
527,Schindler's List (1993),Drama|War,33306.0,4.246577
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,32637.0,3.691577


#Segunda Heurística: Nota média e filtrando votos

In [31]:
filmes.sort_values(by='notas_medias', ascending=False).head(10)

,títulos,generos,total_de_votos,notas_medias
filmeId,,,,
151503,Bloody Reunion (2006),Horror|Mystery|Thriller,1.0,5.0
219201,An Affair (2018),Thriller,1.0,5.0
158404,Nova Zembla (2011),Drama,1.0,5.0
158426,Elvis & Madona (2010),(no genres listed),1.0,5.0
158514,To the Last Man (1933),Western,1.0,5.0
158526,1 Chance 2 Dance (2014),(no genres listed),2.0,5.0
158571,Mickybo and Me (2005),Comedy|Drama,1.0,5.0
158649,La Soif du Monde (2012),Documentary,1.0,5.0
158681,Courage (2009),Children|Drama,1.0,5.0


In [32]:
filmes.query('total_de_votos >= 10').sort_values(by='notas_medias', ascending=False).head(10)

,títulos,generos,total_de_votos,notas_medias
filmeId,,,,
195529,Mushishi: The Shadow That Devours the Sun (2014),Adventure|Animation|Fantasy,15.0,4.533333
171011,Planet Earth II (2016),Documentary,773.0,4.456016
171495,Cosmos,(no genres listed),235.0,4.455319
151767,Monella (1998),Comedy|Romance,11.0,4.454545
231341,Attack on Titan: Chronicle (2020),Action|Adventure|Animation|Fantasy,10.0,4.450000
159817,Planet Earth (2006),Documentary,1145.0,4.442358
170705,Band of Brothers (2001),Action|Drama|War,1093.0,4.434584
318,"Shawshank Redemption, The (1994)",Crime|Drama,48346.0,4.417232
93404,Queen: Days of Our Lives (2011),Documentary,21.0,4.404762


In [33]:
filmes_com_mais_de_50_votos = filmes.query('total_de_votos >= 50')
filmes_com_mais_de_50_votos.sort_values(by='notas_medias', ascending=False).head(10)

,títulos,generos,total_de_votos,notas_medias
filmeId,,,,
171011,Planet Earth II (2016),Documentary,773.0,4.456016
171495,Cosmos,(no genres listed),235.0,4.455319
159817,Planet Earth (2006),Documentary,1145.0,4.442358
170705,Band of Brothers (2001),Action|Drama|War,1093.0,4.434584
318,"Shawshank Redemption, The (1994)",Crime|Drama,48346.0,4.417232
182723,Cosmos: A Spacetime Odissey,(no genres listed),237.0,4.350211
202439,Parasite (2019),Comedy|Drama,4819.0,4.331189
858,"Godfather, The (1972)",Crime|Drama,29625.0,4.324456
220528,Twelve Angry Men (1954),Drama,135.0,4.322222
